***TODO***
- sklearn
- normalized coordinate only appearing for some patients
- dosegrid shape pickle file generation 
- collapse each df into row to begin ML
- NTCP function
- visualization
- meet with DR. to verify parameters



In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import argparse
import re
import datetime

**READS IN PATIENT RESULTS**

In [2]:
def get_directory():
    # code to take in command line argument of Data folder
    parser = argparse.ArgumentParser()
    parser.add_argument('data_directory', type=str)
    args = parser.parse_args()
    return args.data_directory



In [3]:
#DATA_ROOT = get_directory()
#DATA_ROOT = './data'
DATA_ROOT = './results'

In [4]:
# getting all patient
patient_paths = {}
patient_names = []
for name in os.listdir(DATA_ROOT):
    path = os.path.join(DATA_ROOT, name)
    if os.path.isdir(path):
        patient_paths[name]= path
        patient_names.append(name)
print(patient_paths)

{'SB': './results/SB', 'PM': './results/PM', 'PJ': './results/PJ', 'SW': './results/SW', 'PP': './results/PP', 'FJ1': './results/FJ1', 'BY': './results/BY', 'HL': './results/HL', 'BL': './results/BL', 'AJ': './results/AJ', 'BE': './results/BE', 'DA': './results/DA', 'HD': './results/HD', 'BC': './results/BC', 'LJ': './results/LJ', 'FJ': './results/FJ', 'LL': './results/LL', 'LK': './results/LK', 'CT': './results/CT', 'BB2': './results/BB2', 'HM1': './results/HM1', 'WJ': './results/WJ', 'SR1': './results/SR1', 'TC': './results/TC', 'WL': './results/WL', 'VE': './results/VE', 'MM1': './results/MM1', 'PG': './results/PG', 'PJ2': './results/PJ2', 'MK': './results/MK', 'MB': './results/MB', 'DJ': './results/DJ', 'GB': './results/GB', 'DM': './results/DM', 'DJ1': './results/DJ1', 'GV': './results/GV', 'KS': './results/KS', 'HI': './results/HI', 'DE': './results/DE', 'CW': './results/CW', 'IB': './results/IB', 'IP': './results/IP', 'RD': './results/RD', 'MJ1': './results/MJ1', 'TA': './result

In [5]:
def get_result_DFS(patient_names, patient_paths):
    # returns dict with key as patient name and value as df with pa, aorta, vc, heart constraints and then a number of failures
    # prints error message for any patient without all 4
    
    patient_dfs = {} # dict to be returned
    num_failures = 0
    num_success = 0
    too_many_patients = {}
    not_enough_patients = {}
    
    for name in patient_names:
        print('Working on patient',name, '...', end=' ')
        df = pd.read_csv(os.path.join(patient_paths[name], 'patient_results.csv')) # reads patient csv
        df.set_index(df[name], inplace=True) # sets index correctly
        df.drop(name, axis=1, inplace=True) # drops extra
        
        present_organs = {'heart': [], 'pa': [], 'vc': [], 'aorta': []}
        column_headers = []
        bool_arr = np.array([0,0,0,0], dtype=bool)
        
        for organ in df.columns:
            if 'heart' in organ:
                present_organs['heart'].append(organ)
                column_headers.append(organ)
                bool_arr[0] = 1
            elif 'pa_' in organ:
                present_organs['pa'].append(organ)
                column_headers.append(organ)
                bool_arr[1] = 1               
            elif 'vc_' in organ:
                present_organs['vc'].append(organ)
                column_headers.append(organ)
                bool_arr[2] = 1
            elif 'aorta' in organ:
                present_organs['aorta'].append(organ)
                column_headers.append(organ)
                bool_arr[3] = 1

        newdf = df.filter(column_headers, axis=1)
        
        #detecting failures (none present or too many)
        if len(newdf.columns) == 4 and np.count_nonzero(bool_arr) == 4:
            newcols = dict([(val[0],key) for key,val in present_organs.items()])
            newdf.rename(columns=newcols, inplace=True)
            newdf = newdf.reindex(columns=sorted(newdf.columns))
            patient_dfs[name] = newdf
            num_success += 1
            print('Success!\n\n')
        else:
            num_failures += 1
            too_many = []
            none_present = []
            for organ in present_organs:
                if len(present_organs[organ]) == 0:
                    none_present.append(organ)
                elif len(present_organs[organ]) > 1:
                    too_many.append(organ)
                    
            print('Failure')
            if len(none_present) > 0:
                # if no data found for given organ: print out which organ
                print('No data found for organs:', str(none_present)[1:-1])
                not_enough_patients[name] = none_present
            else: 
                # if multiple data found for organ: print out names
                for i in range(len(too_many)):
                    print('Multiple instances of organ', too_many[i], ':', str(present_organs[too_many[i]])[1:-1])
                if name not in too_many_patients:
                    too_many_patients[name] = too_many
            
            print('\n\n')
    
    print('success:',num_success,'--- failure:',num_failures)
    return patient_dfs, num_failures, not_enough_patients, too_many_patients

In [6]:
# work in progress
# def handle_too_many(too_many, patient_paths):
#     # returns unedited DF of patients that failed for having toow many
#     dfs = {}

#     for name in too_many:
#         df = pd.read_csv(os.path.join(patient_paths[name], 'patient_results.csv'))
        
    
#     for name in names:
#         print('working on patient', name)
#         df = pd.read_csv(os.path.join(patient_paths[name], 'patient_results.csv'))
    
#         dfs[name] = df
#     return dfs
    

In [7]:
def handle_too_few(missing_organs, patient_paths):
    dfs = {}
    
    for name in missing_organs:
        df = pd.read_csv(os.path.join(patient_paths[name],'patient_results.csv'))#, index_col=0)
        df.set_index(df[name], inplace=True) # sets index correctly
        df.drop(name, axis=1, inplace=True) # drops extra
        
        for missing in missing_organs[name]:
            df[missing] = pd.Series([0 for x in range(len(df))])
        
        
        present_organs = {'heart': [], 'pa': [], 'vc': [], 'aorta': []}
        for organ in df.columns:
            for o in present_organs:
                if o in organ:
                    present_organs[o].append(organ)
        
        newcols = dict([(val[0],key) for key,val in present_organs.items()])
        df.rename(columns=newcols, inplace=True)
        df = df.reindex(columns=sorted(df.columns))
        df.fillna(0, inplace=True)
        dfs[name] = df
        
    return dfs

In [8]:
patient_dfs, num_fails, failures, patients_with_many = get_result_DFS(patient_names, patient_paths)

Working on patient SB ... Success!


Working on patient PM ... Success!


Working on patient PJ ... Success!


Working on patient SW ... Success!


Working on patient PP ... Success!


Working on patient FJ1 ... Success!


Working on patient BY ... Success!


Working on patient HL ... Success!


Working on patient BL ... Success!


Working on patient AJ ... Success!


Working on patient BE ... Success!


Working on patient DA ... Success!


Working on patient HD ... Failure
No data found for organs: 'vc'



Working on patient BC ... Success!


Working on patient LJ ... Failure
No data found for organs: 'pa'



Working on patient FJ ... Success!


Working on patient LL ... Success!


Working on patient LK ... Success!


Working on patient CT ... Success!


Working on patient BB2 ... Success!


Working on patient HM1 ... Success!


Working on patient WJ ... Failure
No data found for organs: 'pa'



Working on patient SR1 ... Success!


Working on patient TC ... Success!


Working on pati

In [9]:
failure_dfs = handle_too_few(failures, patient_paths)

In [10]:
# failure_dfs['FJ']
# failure_dfs['DM']
# failure_dfs['HI']
failures

{'HD': ['vc'],
 'LJ': ['pa'],
 'WJ': ['pa'],
 'VE': ['pa'],
 'DJ': ['pa'],
 'DJ1': ['pa'],
 'GV': ['pa'],
 'IB': ['pa'],
 'IP': ['pa'],
 'WT': ['pa']}

In [11]:
failure_dfs['LJ']

,aorta,heart,pa,vc
LJ,,,,
max_dose,65.684604648,54.609698088,0.0,5.639171748
max_coord,[ 96 111 108],[ 95 109 104],0.0,[ 83 86 107]
normalized_max_coord,[-0.07979798 0.08508772 0.38410596],[-0.08585859 0.0745614 0.35761589],0.0,[-0.15858586 -0.04649123 0.37748344]
axial_scale_factors,[165 190 151],[165 190 151],0.0,[165 190 151]
v25cc,12.015625,4.203125,0.0,0.0
D0.5cc,60.748336644000005,44.441660556,0.0,4.80144816
D1cc,59.397520463999996,39.016976868,0.0,4.466482176
D4cc,44.456942772,25.447453968,0.0,2.914257588
dist_to_max_from_t10,15.40844375298456,14.399194614823275,0.0,15.834539427747462


In [12]:
patient_dfs['SB']

,aorta,heart,pa,vc
SB,,,,
max_dose,49.95396046000001,52.418956200000004,51.78076592,51.812725240000006
max_coord,[38 78 56],[42 74 53],[48 68 54],[44 72 60]
normalized_max_coord,[-0.3328841 0.02564103 0.40659341],[-0.29514825 0. 0.37362637],[-0.23854447 -0.03846154 0.38461538],[-0.27628032 -0.01282051 0.45054945]
axial_scale_factors,[106 156 91],[106 156 91],[106 156 91],[106 156 91]
v25cc,8.609375,36.046875,1.203125,5.546875
D0.5cc,46.693836880000006,51.37920534,33.89050144,49.08356844
D1cc,44.5418461,50.99478604,26.992745799999998,47.24212812
D4cc,33.4514497,49.1955451,17.472000899999998,33.11304246
dist_to_max_from_t10,12.821080377285542,11.550962942454591,10.898300747532778,12.606181671169532


In [13]:
for name in failure_dfs:
    patient_dfs[name] =failure_dfs[name]

In [14]:
len(patient_dfs.keys())

46

**READS IN CSV OF OTHER PATIENT INFO**

In [15]:
label_data = pd.read_csv('./label_data.csv',index_col =False)
pd.set_option('display.max_columns', None)

important = ['Initials', 
#              'Rx_dose', '# Fractions', 
#              'Dose per Fraction', 
#              'Multiple/Diff Lung Tx DELIVERED (0 N, 1 Y)',
             'Courses Prior to event (#s, As and Bs)',
             'DOB', 'Sex (0M, 1F)', 'Race', 
             'Smoking History', 'Tumor Location (full)',
              'Tumor x and y (cm)', 
              'Tumor z (T)', 
             'Tx Start', 
#              'Tumor Stage', 'Tumor Type (Histology)', 
             'Clinical_Post', 'Event (1 Y, 0 N)',
             'Survival Status (0 living, 1 deceased (date if known))',
            ]
label_data = label_data.filter(important, axis=1)
label_data.set_index(label_data['Initials'], inplace=True)
label_data.drop('Initials', axis=1, inplace=True)

rename = {'Sex (0M, 1F)': 'Sex', 
          'Survival Status (0 living, 1 deceased (date if known))': 'Survival',
          'Courses Prior to event (#s, As and Bs)': 'Prior_Courses',
#           '# Fractions': 'Num_Fractions', 'Dose per Fraction':'DPF',
#           'Multiple/Diff Lung Tx DELIVERED (0 N, 1 Y)':'Other_TX',
          'Tumor Location (full)':'Tumor_Loc', 
           'Tumor x and y (cm)':'X_Y', 
           'Tumor z (T)':'Z', 
#           'Tumor Stage':'Stage', 'Tumor Type (Histology)': 'Type',
          'Smoking History': 'Smoke', 
          'Event (1 Y, 0 N)': 'Event', 'Tx Start': 'Start_TX',
         }

label_data.rename(columns=rename, inplace=True)

In [16]:
label_data.head()

,Prior_Courses,DOB,Sex,Race,Smoke,Tumor_Loc,X_Y,Z,Start_TX,Clinical_Post,Event,Survival
Initials,,,,,,,,,,,,
MM1,1A,3/7/35,1,White,Former(30+),LUL,"(5.161, 4.283)",3.0,5/23/16,Aortic valve replacement (10/18/2016),1,0
WL,1A,11/18/55,1,White,Never,LUL,"(3.540, 1.871)",3.0,4/6/17,Acute on chronic systolic right ventricular fa...,1,1/23/19
HM1,1A,6/1/32,0,African American,Former(30+),LUL,"(5.527, 2.214)",3.5,8/1/16,No developments as of 3/1/2018,0,3/9/18
WJ,1A,1/10/62,0,White,Current,LUL,"(2.593, 2.316)",3.5,11/2/16,NaN,0,0
LJ,2A,3/20/62,0,White,Former(30+),LUL,"(4.243, 4.545)",4.0,3/18/15,No developments as of 7/19/2019,0,0


In [17]:
# label_data.Smoke.value_counts()
# all_patients = list(label_data.index)
# label_data.loc[['LL', 'TC', 'BE', 'CW', 'DJ1', 'HM1', 'LJ']]

In [18]:
# label_data[['Start_TX','End_TX', 'Days_to_event','Survival', 'Event', 'Clinical_Post']]


In [19]:
def extract_date(original):
    # returns latest date in given string
    date_match = r'(([0-9]{1,2})/([0-9]{1,2})/([0-9]{2,4}))'
    matches = re.findall(date_match, original)
    dates = []
    for match in matches:
        month = int(match[1])
        day = int(match[2])
        if len(match[3]) == 2:
            year = int('20'+match[3])
        else:
            year = int(match[3])
            
        dates.append(datetime.date(year, month,day))
        
    if len(dates) == 0:
        return False
    return max(dates)
    
# test = 'Aortic valve replacement (10/18/2016) asfda  1/2/12, jdskfbadskbfa'
# test2= 'sadfd'
# if extract_date(test2):
#     print('yes')
# else:
#     print('no')

In [20]:
def get_event_dates(label_data):
    post = pd.DataFrame(label_data.Clinical_Post)
    post['Event_Date'] = 'default'
    post['Event_Description'] = 'N/A'
    
    for row in post.iterrows():
        text = str(row[1].Clinical_Post).strip().lower()
        if text == 'nan' or text == 'no data':
            row[1].Clinical_Post = 'No Data'
            row[1].Event_Date = 'N/A'
        elif 'no' in text and 'developments' in text:
            date = extract_date(text)
            row[1].Clinical_Post = 'No Developments'
            row[1].Event_Date = 'N/A'
    #         if date:
    #             row[1].Event_Date = str(date)
    #         else:
    #             row[1].Event_Date = 'N/A'
        else:
            date = extract_date(text)
            row[1].Clinical_Post = 'Event'
            row[1].Event_Description = text
            if date:
                row[1].Event_Date = str(date)
            else:
                row[1].Event_Date = 'CHECK'
    
    post['Event'] = label_data.Event
    post.loc[((post.Event == 0) & (post.Clinical_Post == 'Event')), 'Event'] = 1
    
    label_data.drop('Clinical_Post',axis=1, inplace=True)
    label_data['Event'] = post['Event']
    label_data['Event_Date']= post['Event_Date']
    
    return label_data
    
label_data = get_event_dates(label_data)

In [21]:
label_data.Tumor_Loc.value_counts()##
# label_data.info()

LUL          20
LLL          12
RLL           7
RUL           7
L LINGULA     2
LT Hilum      1
RML           1
Name: Tumor_Loc, dtype: int64

In [22]:
def separate_xyz(label_data):
    xyz = pd.DataFrame(label_data.X_Y)
    xyz = xyz.X_Y.str.strip('()').str.split(', ',expand=True)
    xyz.columns = ['X', 'Y']
    xyz['Z'] = label_data.Z

    label_data.drop('X_Y', axis=1, inplace=True)
    label_data.drop('Z', axis=1, inplace=True)
    label_data['X'] = xyz.X.astype(float)
    label_data['Y'] = xyz.Y.astype(float)
    label_data['Z'] = xyz.Z.astype(float)
    
separate_xyz(label_data)

In [23]:
def convert_to_datetime(label_data):
    dates = pd.DataFrame(label_data.DOB)
    dates['Start_TX'] = label_data.Start_TX.copy()
    dates['Survival'] = label_data.Survival.copy()

    for row in dates.iterrows():
        DOB = row[1].DOB
        DOB_lst = DOB.strip().split('/')
        start_date = row[1].Start_TX
        start_lst = start_date.strip().split('/')
        surv = row[1].Survival
        #year,month,day
        row[1].DOB= datetime.date(int('19'+DOB_lst[2]), int(DOB_lst[0]), int(DOB_lst[1]))
        row[1].Start_TX= datetime.date(int('20'+start_lst[2]), int(start_lst[0]), int(start_lst[1]))
        if surv != '0':
            surv_date =  surv.strip().split('/')
            row[1].Survival = datetime.date(int('20'+surv_date[2]), int(surv_date[0]), int(surv_date[1]))
        else:
#             row[1].Survival = 'N/A'
            row[1].Survival
            
    dates['Age_at_TX'] = dates.Start_TX - dates.DOB
    label_data.drop('DOB', axis=1, inplace=True)
    label_data.drop('Start_TX', axis=1, inplace=True)
    label_data.drop('Survival', axis=1, inplace=True)
    label_data['Age_at_TX'] = dates.Age_at_TX
    label_data['Start_TX'] = dates.Start_TX
    label_data['Survival'] = dates.Survival

# dates['Time_TX_to_Event'] = dates[dates.Event_Date != 'N/A'] - dates.Start_TX
convert_to_datetime(label_data)

In [24]:
def convert_prior_courses(label_data):
    prior = pd.DataFrame(label_data.Prior_Courses)
    for row in prior.iterrows():
        row[1].Prior_Courses = int(row[1].Prior_Courses[0])
    
    label_data.drop('Prior_Courses', axis=1, inplace=True)
    label_data['Prior_Courses'] = prior.astype(int)

convert_prior_courses(label_data)
# convert age to age in days
label_data.Age_at_TX = pd.to_numeric(label_data.Age_at_TX.dt.days, downcast='integer')


In [25]:
timelag = []
for i in range(len(label_data)):
    row = label_data.iloc[i]
    
    if row.Event_Date != 'N/A':
        event = datetime.datetime.strptime(row.Event_Date, '%Y-%m-%d').date()
#     elif type(row.Survival) is datetime.date:
#         event = row.Survival
    else:
        event = datetime.datetime.now().date()
    
    
    timelag.append(event - row.Start_TX)

label_data['Timelag'] = timelag
label_data.Timelag = pd.to_numeric(label_data.Timelag.dt.days, downcast='integer')
label_data.head()

,Sex,Race,Smoke,Tumor_Loc,Event,Event_Date,X,Y,Z,Age_at_TX,Start_TX,Survival,Prior_Courses,Timelag
Initials,,,,,,,,,,,,,,
MM1,1,White,Former(30+),LUL,1,2016-10-18,5.161,4.283,3.0,29663,2016-05-23,0,1,148
WL,1,White,Never,LUL,1,2018-06-04,3.540,1.871,3.0,22420,2017-04-06,2019-01-23,1,424
HM1,0,African American,Former(30+),LUL,0,N/A,5.527,2.214,3.5,30742,2016-08-01,2018-03-09,1,1357
WJ,0,White,Current,LUL,0,N/A,2.593,2.316,3.5,20020,2016-11-02,0,1,1264
LJ,0,White,Former(30+),LUL,0,N/A,4.243,4.545,4.0,19356,2015-03-18,0,2,1859


In [26]:
# lst1 = []
# lst2 = []
# for i in range(len(label_data)):
#     if type(label_data.iloc[i].Event_Date) is str and label_data.iloc[i].Event_Date != 'N/A':
#         x = datetime.datetime.strptime(label_data.iloc[i].Event_Date, '%Y-%m-%d')
#     else:
#         x = datetime.datetime.now()
#     y = datetime.datetime.strptime(str(label_data.iloc[i].Start_TX).strip(':0: '), '%Y-%m-%d')
#     lst1.append(x)
#     lst2.append(y)

# label_data.Event_Date = lst1
# label_data.Start_TX = lst2
# label_data['Timelag'] = label_data.Event_Date - label_data.Start_TX
# label_data.Timelag = pd.to_numeric(label_data.Timelag.dt.days, downcast='integer')
# label_data.drop(['Event_Date', 'Start_TX'], axis=1, inplace=True)

In [27]:
# label_data

In [28]:
col_order= ['Sex', 'Race', 'Smoke', 'Tumor_Loc', 
            'Prior_Courses', 'X', 'Y', 'Z', 
            'Age_at_TX', 'Start_TX',
            'Survival', 'Timelag', 
            'Event','Event_Date']

In [29]:
label_data = label_data[col_order]

In [30]:
label_data

,Sex,Race,Smoke,Tumor_Loc,Prior_Courses,X,Y,Z,Age_at_TX,Start_TX,Survival,Timelag,Event,Event_Date
Initials,,,,,,,,,,,,,,
MM1,1,White,Former(30+),LUL,1,5.161,4.283,3.0,29663,2016-05-23,0,148,1,2016-10-18
WL,1,White,Never,LUL,1,3.540,1.871,3.0,22420,2017-04-06,2019-01-23,424,1,2018-06-04
HM1,0,African American,Former(30+),LUL,1,5.527,2.214,3.5,30742,2016-08-01,2018-03-09,1357,0,N/A
WJ,0,White,Current,LUL,1,2.593,2.316,3.5,20020,2016-11-02,0,1264,0,N/A
LJ,0,White,Former(30+),LUL,2,4.243,4.545,4.0,19356,2015-03-18,0,1859,0,N/A
WT,1,African American,Former,LUL,3,4.522,0.258,4.0,17861,2016-08-23,0,1335,0,N/A
CW,1,White,Former(30+),LUL,1,7.123,1.921,4.0,27878,2015-08-03,0,1721,0,N/A
DJ1,0,White,Former(30+),LUL,1,5.401,7.231,4.0,30962,2015-08-13,0,654,1,2017-05-28
PJ2,1,White,Former(30+),LUL,1,4.572,5.149,4.0,27611,2017-07-31,0,993,0,N/A


In [31]:
# label_data.loc[(label_data.index == 'MM1')]

In [32]:
patients_in_spreadsheet = set(label_data.index)
patients_in_results = set(patient_dfs.keys())
# patients_in_spreadsheet-patients_in_results

In [33]:
patients_without_results = patients_in_spreadsheet-patients_in_results

In [34]:
patients_without_results

{'BB1', 'BJ', 'HT', 'WP'}

In [35]:
label_data = label_data.loc[patients_in_results,:]
label_data.head()

,Sex,Race,Smoke,Tumor_Loc,Prior_Courses,X,Y,Z,Age_at_TX,Start_TX,Survival,Timelag,Event,Event_Date
Initials,,,,,,,,,,,,,,
CW,1,White,Former(30+),LUL,1,7.123,1.921,4.0,27878,2015-08-03,0,1721,0,N/A
HM1,0,African American,Former(30+),LUL,1,5.527,2.214,3.5,30742,2016-08-01,2018-03-09,1357,0,N/A
KS,1,White,Current,L LINGULA,1,10.655,9.200,9.5,24878,2018-10-17,0,550,0,N/A
MB,1,White,Former,LUL,1,5.684,3.735,4.0,24824,2015-08-05,2015-10-15,1719,0,N/A
LL,1,White,Former(30+),RLL,1,-0.994,5.842,10.0,25023,2015-09-21,0,1086,1,2018-09-11


**Combine data**

In [36]:
def build_single_df(df, patient_name, new_cols, stats):
    patient_vals = np.array([df.loc[stats,:].values.flatten('F')]).astype(float)
    df = pd.DataFrame(data=patient_vals, columns=new_cols)
    df.index = [patient_name]
    return df

In [37]:
def build_data_df(patient_dfs):
    organs = ['aorta', 'heart', 'pa', 'vc']
    stats = ['max_dose', 'v25cc', 'D0.5cc', 'D1cc', 'D4cc']#, 'Dmax_NTCP', 'V25_NTCP', 'D0.5_NTCP', 'D1_NTCP', 'D4_NTCP']
    
    new_cols = []
    for organ in organs:
        for s in stats:
#       print(organ +'_'+s)
            new_cols.append(organ+'_'+s)
    new_cols = np.array(new_cols)
    
    final_df = pd.DataFrame(columns=new_cols)
    for name in patient_dfs:
        new_df = build_single_df(patient_dfs[name], name, new_cols, stats)
        final_df = pd.concat([final_df, new_df])
    
    
    return final_df

In [38]:
compiled_patient_df = build_data_df(patient_dfs)
compiled_patient_df

,aorta_max_dose,aorta_v25cc,aorta_D0.5cc,aorta_D1cc,aorta_D4cc,heart_max_dose,heart_v25cc,heart_D0.5cc,heart_D1cc,heart_D4cc,pa_max_dose,pa_v25cc,pa_D0.5cc,pa_D1cc,pa_D4cc,vc_max_dose,vc_v25cc,vc_D0.5cc,vc_D1cc,vc_D4cc
SB,49.953960,8.609375,46.693837,44.541846,33.451450,52.418956,36.046875,51.379205,50.994786,49.195545,51.780766,1.203125,33.890501,26.992746,17.472001,51.812725,5.546875,49.083568,47.242128,33.113042
PM,28.732152,1.109375,25.850038,25.095479,23.411566,34.984006,3.031250,30.565427,28.326365,24.300617,26.152038,0.031250,16.803396,16.127666,14.379898,1.668209,0.000000,0.986869,0.807382,0.465408
PJ,15.469016,0.000000,14.999235,14.731580,13.677716,61.912864,72.625000,57.529470,54.681478,46.173572,8.723881,0.000000,7.374236,6.642106,3.133113,10.071997,0.000000,7.071999,5.994975,3.183804
SW,30.286425,0.578125,25.529823,22.420834,15.632079,55.938310,6.687500,48.636755,43.298575,29.370787,3.902508,0.000000,3.664320,1.626005,1.626005,4.391118,0.000000,4.128448,4.006655,0.818562
PP,33.350184,13.656250,29.235274,28.623298,27.382640,11.851033,0.000000,11.125596,10.743319,8.823692,55.713916,7.921875,52.774943,48.256040,33.223953,54.177564,23.562500,52.603757,51.688449,46.112834
FJ1,19.816527,0.000000,16.599815,15.167541,11.408231,48.365220,4.750000,40.415931,35.792198,26.686368,0.596838,0.000000,0.460546,0.342514,0.342514,19.675293,0.000000,15.255381,11.839691,1.160079
BY,20.928470,0.000000,19.884291,19.036579,15.350278,34.933725,3.359375,31.435727,29.483974,24.231306,1.168091,0.000000,0.781909,0.627309,0.574071,8.000253,0.000000,6.156426,4.094441,0.773714
HL,9.055674,0.000000,8.224925,7.946645,7.250176,49.881474,12.546875,44.228752,40.957360,32.539452,31.535676,0.421875,23.727302,20.997546,14.004814,17.177190,0.000000,15.696753,14.367638,10.099653
BL,47.081517,2.968750,37.097524,32.910155,23.315160,60.918565,13.625000,48.882108,44.108267,32.764580,28.063299,1.390625,26.432867,25.430629,22.578442,11.697476,0.000000,4.328503,2.095525,0.506235
AJ,44.559248,10.718750,39.269301,36.967356,31.372219,52.500380,47.265625,51.523756,51.169825,49.582840,51.934535,2.656250,47.527583,41.955845,18.865337,51.651730,9.453125,50.546994,49.951595,45.388621


In [39]:
patient_dfs['BB2']

,aorta,heart,pa,vc
BB2,,,,
max_dose,23.93839817,46.19424954,47.786215500000004,28.18656424
max_coord,[54 94 76],[62 83 77],[64 80 70],[56 86 77]
normalized_max_coord,[-0.25553097 -0.01680108 0.31782946],[-0.18473451 -0.07594086 0.3255814 ],[-0.1670354 -0.09206989 0.27131783],[-0.23783186 -0.05981183 0.3255814 ]
axial_scale_factors,[113 186 129],[113 186 129],[113 186 129],[113 186 129]
v25cc,0.0,11.578125,6.171875,0.546875
D0.5cc,22.62147119,39.43550687,40.59282096,25.05492798
D1cc,22.14179188,35.62020314999999,36.14163569,24.24456957
D4cc,20.76724553,29.904583350000006,27.60058138,21.56231088
dist_to_max_from_t10,12.561178413070964,12.245614648722212,10.823964298028704,12.772116235182015


In [40]:
final_df = pd.concat([label_data, compiled_patient_df], axis=1)
# final_df.drop(['Survival', 'Event_Date', 'Start_TX'], axis=1, inplace=True)
final_df.drop(['Survival', 'Event_Date', 'Start_TX'], axis=1, inplace=True)
final_df

,Sex,Race,Smoke,Tumor_Loc,Prior_Courses,X,Y,Z,Age_at_TX,Timelag,Event,aorta_max_dose,aorta_v25cc,aorta_D0.5cc,aorta_D1cc,aorta_D4cc,heart_max_dose,heart_v25cc,heart_D0.5cc,heart_D1cc,heart_D4cc,pa_max_dose,pa_v25cc,pa_D0.5cc,pa_D1cc,pa_D4cc,vc_max_dose,vc_v25cc,vc_D0.5cc,vc_D1cc,vc_D4cc
CW,1,White,Former(30+),LUL,1,7.123,1.921,4.0,27878,1721,0,50.649872,12.078125,43.308904,40.249987,31.864126,57.985718,7.781250,47.619239,42.941155,31.131166,59.937246,7.015625,56.169084,52.084464,34.357566,17.012399,0.000000,15.507949,14.838689,8.396051
HM1,0,African American,Former(30+),LUL,1,5.527,2.214,3.5,30742,1357,0,52.931055,40.187500,51.766538,51.584367,50.378509,27.666470,0.265625,23.276036,21.422875,16.897081,25.142346,0.015625,18.454275,16.309856,8.346141,15.130018,0.000000,14.195834,13.805326,12.013205
KS,1,White,Current,L LINGULA,1,10.655,9.200,9.5,24878,550,0,19.277748,0.000000,18.112433,17.587820,15.555017,37.910853,2.328125,31.193862,28.138905,22.510150,7.424156,0.000000,5.437350,4.840003,1.273314,3.185763,0.000000,2.866407,2.692640,1.623593
MB,1,White,Former,LUL,1,5.684,3.735,4.0,24824,1719,0,43.429293,4.484375,35.826457,32.779170,25.677309,56.185619,1.187500,31.539300,26.600004,16.175898,45.830279,0.828125,28.620957,23.761683,13.592739,6.813677,0.000000,6.409245,6.232062,5.417848
LL,1,White,Former(30+),RLL,1,-0.994,5.842,10.0,25023,1086,1,0.094535,0.000000,0.086249,0.086249,0.086249,1.090614,0.000000,0.754907,0.658258,0.477264,13.725299,0.000000,8.966031,6.849898,3.434567,29.529727,0.890625,26.040380,24.561707,20.574073
TC,0,African American,Current,RLL,1,-4.008,11.378,10.0,27892,943,0,9.730686,0.000000,7.849323,7.281639,6.044379,49.093749,14.328125,45.694923,42.336126,34.745172,1.335513,0.000000,0.942501,0.786024,0.542211,42.361599,1.093750,29.756103,25.491195,14.748867
BE,1,White,Former(30+),LLL,1,4.044,2.661,4.0,32103,2974,0,66.510848,4.000000,50.690934,42.040150,24.811509,12.398851,0.000000,7.819625,5.476514,1.522543,1.690538,0.000000,1.045227,0.911218,0.632209,8.234732,0.000000,7.709190,7.302297,2.832789
PJ2,1,White,Former(30+),LUL,1,4.572,5.149,4.0,27611,993,0,51.230228,37.218750,50.248823,49.898323,47.055803,52.369948,31.937500,51.143249,50.530018,46.615139,53.604116,24.343750,52.891329,52.327772,49.982908,23.646790,0.000000,21.509893,20.671877,16.810428
GV,1,White,Former,RLL,1,-3.765,6.468,11.0,28582,253,1,15.761207,0.000000,13.434130,12.582429,8.806946,60.471948,19.859375,54.664863,52.939959,44.238954,0.000000,0.000000,0.000000,0.000000,0.000000,55.076543,2.078125,41.668573,35.687942,16.065796
DJ1,0,White,Former(30+),LUL,1,5.401,7.231,4.0,30962,654,1,47.951575,13.125000,43.001292,40.676977,33.684796,17.233691,0.000000,11.774891,10.632706,8.011479,0.000000,0.000000,0.000000,0.000000,0.000000,13.932511,0.000000,12.416282,11.789049,9.086258


In [41]:
final_df.to_csv('classification_data.csv')